<a href="https://colab.research.google.com/github/DavidbigC/Perp-arb-calculator/blob/main/The_MM_%26_Perp_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title The MM & Perp Calculator

import ipywidgets as widgets
from IPython.display import display


def calculate_arbitrage_allocation(current_price, ltv, long_liq_price, short_liq_price, liquidation_factor):
    # Calculate leverage for long position using liquidation factor
    long_leverage = 1 / (1 - liquidation_factor * (long_liq_price / current_price))

    # Calculate leverage for short position using liquidation factor
    short_leverage = 1 / ((short_liq_price / current_price) - 1)

    # Calculate capital allocation
    total_leverage = long_leverage + short_leverage
    long_allocation = short_leverage / total_leverage
    short_allocation = long_leverage / total_leverage

    # Calculate capital efficiency
    capital_efficiency = long_allocation * long_leverage

    return {
        'long_allocation': long_allocation,
        'short_allocation': short_allocation,
        'long_leverage': long_leverage,
        'short_leverage': short_leverage,
        'capital_efficiency': capital_efficiency
    }

# Create widgets
current_price_widget = widgets.FloatText(
    value=1000,
    description='Current Price:',
    style={'description_width': 'initial'}
)

ltv_widget = widgets.FloatSlider(
    value=0.6666,
    min=0,
    max=1.0,
    step=0.01,
    description='LTV:',
    readout_format='.4f'
)

liquidation_factor_widget = widgets.FloatSlider(
    value=0.6666,  # Default to the same as LTV
    min=0,
    max=1.0,
    step=0.01,
    description='Liquidation Factor:',
    readout_format='.4f',
    style={'description_width': 'initial'}
)

long_liq_price_widget = widgets.FloatText(
    value=800,
    description='Long Liq Price:',
    style={'description_width': 'initial'}
)

short_liq_price_widget = widgets.FloatText(
    value=1200,
    description='Short Liq Price:',
    style={'description_width': 'initial'}
)

output = widgets.Output()

# Add widget for total investment
total_investment_widget = widgets.FloatText(
    value=10000,
    description='Total Investment ($):',
    style={'description_width': 'initial'}
)

# Add widget for swap fee
swap_fee_widget = widgets.FloatSlider(
    value=0.003,  # 0.3% default
    min=0,
    max=0.01,
    step=0.0001,
    description='Swap Fee %:',
    readout_format='.4f',
    style={'description_width': 'initial'}
)

def on_value_change(change):
    with output:
        output.clear_output()
        try:
            result = calculate_arbitrage_allocation(
                current_price_widget.value,
                ltv_widget.value,
                long_liq_price_widget.value,
                short_liq_price_widget.value,
                liquidation_factor_widget.value
            )

            # Calculate dollar amounts for each position
            long_amount = total_investment_widget.value * result['long_allocation']
            short_amount = total_investment_widget.value * result['short_allocation']

            # Calculate token quantity and fees for long position
            long_notional = long_amount * result['long_leverage']
            long_token_quantity = long_notional / current_price_widget.value
            long_swap_fee = long_notional * swap_fee_widget.value

            # Calculate fees for short position (0.01% + $1)
            short_notional = short_amount * result['short_leverage']
            short_fee = (short_notional * 0.0001) + 1

            print("\nResults:")
            print(f"Long position: Allocate {result['long_allocation']*100:.2f}% of capital (${long_amount:,.2f}) with {result['long_leverage']:.2f}x leverage")
            print(f"Long Position Token Quantity: {long_token_quantity:.4f} tokens")
            print(f"Long Position Swap Fee: ${long_swap_fee:.2f}")
            print(f"\nShort position: Allocate {result['short_allocation']*100:.2f}% of capital (${short_amount:,.2f}) with {result['short_leverage']:.2f}x leverage")
            print(f"Short Position Fee: ${short_fee:.2f}")
            print(f"\nTotal Transaction Fees: ${(long_swap_fee + short_fee):.2f}")
            print(f"Capital Efficiency: {result['capital_efficiency']:.2f}x")

            # Calculate and display notional exposure
            print(f"\nNotional Exposure:")
            print(f"Long Exposure: ${long_notional:,.2f}")
            print(f"Short Exposure: ${short_notional:,.2f}")

        except Exception as e:
            print(f"Error: {str(e)}")

# Observe changes in ALL widgets
current_price_widget.observe(on_value_change, names='value')
ltv_widget.observe(on_value_change, names='value')
liquidation_factor_widget.observe(on_value_change, names='value')
long_liq_price_widget.observe(on_value_change, names='value')
short_liq_price_widget.observe(on_value_change, names='value')
total_investment_widget.observe(on_value_change, names='value')
swap_fee_widget.observe(on_value_change, names='value')

# Trigger initial calculation
on_value_change({'new': None})

# Display all widgets
display(
    current_price_widget,
    ltv_widget,
    liquidation_factor_widget,
    long_liq_price_widget,
    short_liq_price_widget,
    total_investment_widget,
    swap_fee_widget,
    output
)

FloatText(value=1000.0, description='Current Price:', style=DescriptionStyle(description_width='initial'))

FloatSlider(value=0.6666, description='LTV:', max=1.0, readout_format='.4f', step=0.01)

FloatSlider(value=0.6666, description='Liquidation Factor:', max=1.0, readout_format='.4f', step=0.01, style=S…

FloatText(value=800.0, description='Long Liq Price:', style=DescriptionStyle(description_width='initial'))

FloatText(value=1200.0, description='Short Liq Price:', style=DescriptionStyle(description_width='initial'))

FloatText(value=10000.0, description='Total Investment ($):', style=DescriptionStyle(description_width='initia…

FloatSlider(value=0.003, description='Swap Fee %:', max=0.01, readout_format='.4f', step=0.0001, style=SliderS…

Output()